## Monte carlo methods


The first thoughts and attempts I made to practice [the Monte Carlo Method] were suggested by a question which occurred to me in 1946 as I was convalescing from an illness and playing solitaires. The question was what are the chances that a Canfield solitaire laid out with 52 cards will come out successfully? After spending a lot of time trying to estimate them by pure combinatorial calculations, I wondered whether a more practical method than "abstract thinking" might not be to lay it out say one hundred times and simply observe and count the number of successful plays. This was already possible to envisage with the beginning of the new era of fast computers, and I immediately thought of problems of neutron diffusion and other questions of mathematical physics, and more generally how to a processes described by certain differential equations into an equivalent form interpretable as a succession of random operations. Later [in 1946], I described the idea to John von Neumann, and we began to plan actual calculations.[14]

Being secret, the work of von Neumann and Ulam required a code name.[15] A colleague of von Neumann and Ulam, Nicholas Metropolis, suggested using the name Monte Carlo, which refers to the Monte Carlo Casino in Monaco where Ulam's uncle would borrow money from relatives to gamble.

## Standard option pricing based on brownian motion

### Problem

We will try to consider an asset, say 1 stock in Google.

Say we wanted to have the option to buy the Google stock 1 year from now at a price we set today.

What is this option worth?

#### Formalism

Let the following variables denote
- $r$ the intrest rate
- $K$ the fixed price
- $t$ timepoint today
- $T$ time in a year
- $S_t$ price of the Google stock today



\begin{equation*}
\text{Value of option today} = V(S,K,r)  =  e^{-rT}[S_T-K]
\end{equation*}

We create an Euler aproximation storing the skeleton of the price path at 52 points (1 for each week).

\begin{equation*}
  S(t_i) = S(t_{i-1})e^{(r-1/2 \sigma^2)\Delta t_i + \sigma \sqrt{\Delta t_i} N(0,1)}, \quad i=1,...,p\end{equation*}

We calculate the price using the simulated mean as expectation

\begin{equation*}
\mathbb{E}[V] =  \frac{1}{R}\sum_{j=1}^Re^{-rT}\left[ S_T - K \right]^+  
\end{equation*}

## Looking at these concepts in 

<img src="https://www.python.org/static/community_logos/python-logo-master-v3-TM.png">

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import plotly_express as px

import pandas as pd

from plotly.offline import iplot,init_notebook_mode

init_notebook_mode

init_notebook_mode(connected=True)

<img src="https://raw.githubusercontent.com/tqdm/tqdm/master/images/logo.gif">

In [5]:
repetitions = 1000

start_price = 100

intrest = 0.05
volatility = 0.1
timestep = 0.1
end_time = 10
timesteps = int(end_time/timestep)
size = (timesteps,repetitions)
S = np.ones(size)*100
N = np.random.normal(size=size)

for experiment in tqdm(range(repetitions)):
    for time in range(timesteps-1):
        growth = np.exp (
            ( intrest -0.5* volatility**2 ) * timestep +
            volatility * np.sqrt ( timestep ) * N [ time, experiment ]
        )
        S [time +1,experiment] = S [time ,experiment] * growth


100%|██████████| 1000/1000 [00:00<00:00, 2778.88it/s]


In [6]:
df = pd.DataFrame(S)
df["time"] = list(df.index)
pre_fix = "stock_trial_"
df.columns = [pre_fix + str(x) if x!="time" else "time" for x in df.columns]
df = pd.melt(df, id_vars=['time'], value_vars=[pre_fix + x for x in ['1', '2','3','4']])

fig = px.line(df, x="time", y="value", color="variable", line_group="variable", hover_name="variable",
        line_shape="spline", render_mode="svg")
fig.show()

In [25]:
price = np.exp(-intrest*timesteps) * (np.maximum(S[-1,:]-110,0)).mean()
price

0.3818374252469777

## Exercise - Cholesky decomposition

\begin{align}
  A &= L L^T = \begin{pmatrix}
  L_{11} & 0 & 0 \\
  L_{21} & L_{22} & 0 \\
  L_{31} & L_{32} & L_{33}
  \end{pmatrix} \begin{pmatrix}
  L_{11} & L_{21} & L_{31} \\
  0 & L_{22} & L_{32} \\
  0 & 0 & L_{33}
  \end{pmatrix}  \\
  &= \begin{pmatrix}
  L_{11}^2 & -- & \text{symmetric} \\
  L_{21}L_{11} & L_{21}^2+L_{22}^2 & -- \\
  L_{31}L_{11} & L_{31}L_{21} + L_{32}L_{22} & L_{31}^2+L_{32}^2+L_{33}^2
  \end{pmatrix}
\end{align}

This leads to the algorithm for L

\begin{align}
  L_{jj} &= \sqrt{A_{jj} - \sum_{k=1}^{j-1} L_{j,k}^2  } \\
  L_{i,j}&= \frac{1}{L_{jj}}\left(  A_{i,j} - \sum_{k=1}^{j-1} L_{i,k}L_{j,k}  \right), \; \text{for } i>j.
\end{align}

Imprement a function that takes a $nxn$ matrix and returns its cholesky decomposition

In [28]:
def cholesky(x):
    """
    Derive the cholesky decomposition for a given nxn matrix
    args:
        x: nxn matrix
    """
    # Your code
    return L


In [27]:
m = [25, 15, -5, 15, 18, 0, -5, 0, 11]
m = np.reshape(m, (3,3))
m

array([[25, 15, -5],
       [15, 18,  0],
       [-5,  0, 11]])

In [30]:
#cholesky(m)

#### Validate using the numpy equivalent

In [24]:
np.linalg.cholesky(m)

array([[ 5.,  0.,  0.],
       [ 3.,  3.,  0.],
       [-1.,  1.,  3.]])

## The asian option

Now instead lets assume that we feel that Google will perform quite well the next year overall, but maybe not at the particular point in time that is 365 days from now. So we want to be payed alot if the stock on average perfoms well. 

Lets say we want to be payed if Google on average is above 110 the next year. What should I pay today for this bet?

This is the same as pricing an asian option

\begin{equation*}
\text{Value of option today} = V(S,K,r)  =  e^{-rT}[\bar{S}-K], \quad \bar{S}= \frac{1}{m}\sum_{i=1}^m S(t_i) 
\end{equation*}


We create an Euler aproximation storing the skeleton of the price path at 52 points

\begin{equation*}
  S_j(t_i) = S_j(t_{i-1})e^{(r-1/2 \sigma^2)\Delta t_i + \sigma \sqrt{\Delta t_i} N(0,1)}, \quad i=1,...,p, \; j=1,..,R
\end{equation*}

We calculate the price using the simulated mean as expectation

\begin{equation*}
\mathbb{E}[V(S,K,r)] =  \frac{1}{R}\sum_{j=1}^Re^{-rT}\left[\frac{1}{m}\sum_{i=1}^p S_j(t_i) - K \right]^+ 
\end{equation*}


### Lets assume that the variance of the Google stock shifts over time

We will use the fractional Brownian Motion, which has a variance structure modelled using the `Hurst parameter` 

We have an asset $S$ with value $S_t$ at time $t$ denoted as
\begin{align}
  S_t &= S_0 e^{X^h_t}
\end{align}

We consider $S$ at 12 time points. The development of $S$ is governed by a brownian motion $X$ with covariance determined by $h$ called the Hurst parameter 

\begin{align}
  \begin{pmatrix}
  X_1 \\ \vdots \\ X_{12}
  \end{pmatrix} &\sim N_{12}(0,\Sigma) \sim Y \cdot C \\
  C C^T&= \Sigma \\
  Y &\sim N_{12}(0,1) \\
  \Sigma_{t,s} &= \frac{\sigma}{2} \left[ |t|^{2h}+ |s|^{2h}-|t-s|^{2h}  \right]  \\
  \sigma &= \left\lbrace \frac{1}{2} \left[ |p|^{2h}+ |p|^{2h}-|p-p|^{2h}  \right]   \right \rbrace^{-1}, \; p=12\\
  E[B_h(t)B_h(s)] &= \frac{1}{2}(|t|^{2h}+|s|^{2h}-|t-s|^{2h}) \\
 \end{align}

### Exercise

Derive the price of an asian option on an asset driven by a fractional Brownian Motion

For simplicity we will say that

- $r= 0$ 
- $S_t= 1$ 
- $K=2$
Thus we get an asian option with the payout

\begin{align}
(\frac{1}{12}(e^{X(1)} + ··· + e^{X(12)}) - K)^{+}
\end{align}

Where $X(t)$ is the fractional brownian motion which drives the asset price  

In [181]:
K=2
R = 10**5 # Iterations
N = np.random.normal(size = (R,12))

In [31]:
def covar(t,h):
    """
    Function to derive Hurst covariance matrix
    args:
        t: number of timesteps to be modelled with the Hurst covariance
        h: Hurst parameter - h in (0,1)
    """
    return C


In [32]:
def asian_price(t,h,K,N):
    """
    Derive the price for the asian option.
    - Use the covariance function to get sigma
    - Use cholecky factorization to get C
    - Use matrix multiplication to get the covariance structure into the random variable
        drawn from the standard normal distribution
    - Take the average of each price path
    - Subtract K
    - Cut out any negative outcomes
    - Average the payoffs to get a price
    args:
        t: number of timesteps to be modelled with the Hurst covariance
        h: Hurst parameter - h in (0,1)
        K: Price our stock should be above (Strike)
        N: Standard random normal with shape txR, where R is the number of repetitions 
    """
    
    return p
